In [7]:
import pandas as pd
from datetime import datetime
from datetime import date
import numpy as np

In [8]:
df = pd.read_parquet('../do_not_commit/Datasets/Persist_PC_BIOS_DATA.pq')


# bios_release_date_age


In [ ]:
# Number of days since the BIOS version was released.
# Calculate this from the 'ReleaseDate00'.
# Take the .max() 'ReleaseDate00' to account for duplicate records for machineID 1681622.
bios_release_date = df.groupby(['MachineID', 'RWB_EFFECTIVE_DATE'])['ReleaseDate00'].agg('max').reset_index().rename(columns={'ReleaseDate00':'BIOS Release Date'})
bios_release_date

In [11]:
def calculate_age_in_days(target_date_str, in_date):

    # Convert target_date_str to a Python datetime object
    target_date = datetime.strptime(target_date_str, '%Y-%m-%d').date()

    # Calculate the age in days
    age_in_days = (in_date - target_date).days

    return age_in_days


def create_bios_features(in_df):

    # Change effective date to type date
    in_df['BIOS Release Date'] = pd.to_datetime(in_df['BIOS Release Date']).dt.strftime('%Y-%m-%d')

    # Get age in days
    todaydate = date(2023, 7, 4)
    in_df['BIOSReleaseAge'] = in_df['BIOS Release Date'].apply(lambda x: calculate_age_in_days(x, todaydate)
    if pd.notnull(x) else np.nan)
    return in_df

In [12]:
df = create_bios_features(bios_release_date)

In [14]:
df.to_parquet('../do_not_commit/FeatureDatasets/pc_bios_data.pq')